Computes the null model statistics

In [1]:
using Pkg
Pkg.activate("../../env/.")

  Activating project at `~/Documents/1Projects/tracking_cycles_code/env`


In [2]:
include("../../src/analogous_bars.jl")
include("../../src/Eirene_var.jl")
using .analogous_bars
using .Eirene_var
using DelimitedFiles
using HDF5
using Plots
using FileIO
using Random

┌ Warning: Error requiring `HDF5` from `Plots`
│   exception = (LoadError("/Users/irisyoon/.julia/packages/Plots/iYDwd/src/backends/hdf5.jl", 37, UndefVarError(:Group)), Union{Ptr{Nothing}, Base.InterpreterIP}[Ptr{Nothing} @0x000000010b356b1f, Ptr{Nothing} @0x000000010b4003cf, Ptr{Nothing} @0x000000010b40157b, Ptr{Nothing} @0x000000010b3ffae6, Ptr{Nothing} @0x000000010b4000dc, Base.InterpreterIP in top-level CodeInfo for Plots._hdf5_implementation at statement 4, Ptr{Nothing} @0x000000010b41b15f, Ptr{Nothing} @0x000000010b41b2f5, Ptr{Nothing} @0x000000010b41af02, Ptr{Nothing} @0x000000010b41af02, Ptr{Nothing} @0x000000010b41c1d5, Ptr{Nothing} @0x00000001225fe7da, Ptr{Nothing} @0x000000010b3e8160, Ptr{Nothing} @0x000000012263df45, Ptr{Nothing} @0x00000001221c9c85, Ptr{Nothing} @0x000000010b3e8160, Ptr{Nothing} @0x000000010b3f4f4a, Ptr{Nothing} @0x000000010c928d8a, Ptr{Nothing} @0x000000010b3e8160, Ptr{Nothing} @0x000000010b40159f, Ptr{Nothing} @0x000000010b3ffae6, Ptr{Nothing} @0x000000

# Load data

In [ ]:
using Distributed
addprocs(5)
@everywhere include("../../../src/analogous_stats.jl")
@everywhere using .analogous_stats

In [7]:
# load distances
total_D = readdlm("total_dissimilarity.csv")

# get number of neurons in AL
AL_txt = open("AL_neurons_idx.txt", "r")
AL_linebyline = readlines(AL_txt)
close(AL_txt)
n_AL = size(AL_linebyline, 1)

# get number of neurons in V1
V1_txt = open("V1_neurons_idx.txt", "r")
V1_linebyline = readlines(V1_txt)
close(V1_txt)
n_V1 = size(V1_linebyline, 1)

# Define submatrices 
D_V1 = total_D[1:n_V1, 1:n_V1]
D_AL = total_D[n_V1+1:end, n_V1+1:end]
D_V1_AL = total_D[1:n_V1, n_V1+1:end]
    # rows (landmarks): V1
    # columns (witness) : AL
D_AL_V1 = total_D[n_V1+1:end, 1:n_V1];
    # rows (landmarks): AL
    # columns (witness) : V1

41×53 Matrix{Float64}:
 0.424699  0.411983  0.444443  0.575124  …  0.30476   0.374187  0.381591
 0.607417  0.832001  0.576374  0.644771     0.640732  0.809459  0.533623
 0.866342  0.761677  0.685496  0.856879     0.688428  0.745529  0.832457
 0.795257  0.956038  0.626893  0.849059     0.757421  0.952889  0.802012
 0.695159  0.762696  0.491192  0.654693     0.581769  0.789278  0.611313
 0.815913  0.877328  0.552313  0.817288  …  0.662579  0.818494  0.786352
 0.576197  0.7669    0.365874  0.668136     0.49702   0.716563  0.559443
 0.892061  0.803853  0.727002  0.959489     0.729597  0.738541  0.931111
 0.547611  0.291893  0.510774  0.693142     0.247919  0.189238  0.50727
 0.953746  0.819153  0.725814  0.960114     0.69568   0.708533  0.961894
 ⋮                                       ⋱  ⋮                   
 0.450087  0.786944  0.523222  0.676305     0.621632  0.756245  0.504729
 0.945825  0.792079  0.76876   0.960825     0.735642  0.711439  0.953876
 0.354884  0.253876  0.405666  0.5899

In [8]:
# compute
C_V1 = eirene(D_V1, record = "all", maxdim = 1)
C_AL = eirene(D_AL, record = "all", maxdim = 1)
W_V1 = ext_var.compute_Witness_persistence(D_V1_AL)

barcode_V1 = barcode(C_V1, dim = 1)
barcode_AL = barcode(C_AL, dim = 1)
barcode_W_V1_AL = barcode(W_V1["eirene_output"], dim = 1);

In [7]:
p1 = ext_var.plot_PD(barcode_V1, title = "PD(VR(V1))", titlefontsize = 15)
p2 = ext_var.plot_PD(barcode_W_V1_AL, title = "PD(W(V1, AL))")
p3 = ext_var.plot_PD(barcode_AL, title = "PD(VR(AL))", titlefontsize = 15)
plot(p1, p2, p3, layout = grid(1,3))

<!DOCTYPE html>
 
 
 Plots.jl

# Compute null model stats

In [ ]:
start_idx = collect(1:5:200)
for idx in start_idx
    shuffles = collect(idx:idx+4) 
    t = @distributed for k in shuffles
        output_filename = "null_model_stats/" * string(k)
        _, _, _, _, _ = analogous_stats.compute_analogous_bars_null_model(;
                                                                    VR_P = C_V1,
                                                                    D_P = D_V1,
                                                                    VR_Q = C_AL,
                                                                    D_Q = D_AL,
                                                                    P_null_dim = 1,
                                                                    Q_null_dim = 1,
                                                                    output_filename = output_filename
                                                                    )
    

    end
    fetch(t)
end

# Load stats

In [3]:
include("../../../src/analogous_stats.jl")
using .analogous_stats

In [9]:
n_shuffles = 200
output_dir = "null_model_stats/"
selected_V1 = selected_AL = [1, 2]
stats_matrix, stats_matrix_total, selected_bar_count, no_Ftau_P_or_Q, valid_files = parse_null_model_stats(output_dir, 
                                                                         barcode_V1, 
                                                                         barcode_AL,
                                                                         selected_V1,
                                                                         selected_AL,
                                                                         n_shuffles = n_shuffles)

prob_matrix = stats_matrix ./ n_shuffles

2×2 Matrix{Float64}:
 0.0  0.0
 0.0  0.0

In [10]:
stats_matrix_total

2×2 Matrix{Float64}:
 0.0  0.0
 0.0  0.0

In [11]:
stats_matrix

2×2 Matrix{Float64}:
 0.0  0.0
 0.0  0.0